In [1]:
import os
import glob
import json
import numpy as np
import copy
import re

In [2]:
def get_relation_mat(data):
    N=len(data_t)
    r  = re.compile('^http[s]*://[ru.]*bookimed.com(?P<path>/[\w/=-]*)')
    mat = np.zeros((N,N))
    for i in range(0,N):
        for j in range(i+1,N):
            try:
                ref = re.match(r, data_t[j]['referrer']).groupdict()['path'] 
            except:
                ref = data_t[j]['referrer']
            if ref==data_t[i]['path']:
                mat[i,j] = 1
                continue

            if data_t[j]['path']==data_t[i]['path'] and data_t[j]['referrer']==data_t[i]['referrer']:
                if data_t[j]['type']==3:
                    mat[i,j] = 2
                    continue
                if data_t[j]['type']==data_t[i]['type'] and data_t[i]['type']==1:
                    d1 = copy.deepcopy(data_t[i])
                    d2 = copy.deepcopy(data_t[j])
                    #d1 = data_t[i].values()
                    #d2 = data_t[j].values()
                    if d1.pop('timestamp')!=d2.pop('timestamp'):
                        d1.pop('userId')
                        d2.pop('userId')
                        try:
                            d1.pop('originalTimestamp')
                            d2.pop('originalTimestamp')
                        except:
                            pass
                        if d1==d2:
                            mat[i,j]=3
                            continue
                if data_t[j]['type']==2:
                    if data_t[j]['event']==2:
                        mat[i,j]=4
                        continue
                    if data_t[j]['event']==3:
                        mat[i,j]=5
                        continue
                    if data_t[j]['event']==1:
                        mat[i,j]=6
                        continue
    return mat

In [3]:
def get_paths(mat_t):
    N = mat_t.shape[0]
    b = np.ones(N)
    paths = []
    for j in range(N-1,0,-1):
        i=N-1
        path = []
        #print
        while (i>0) and (j>=1):
            #print j
            for i in range(j-1,-1,-1):
                #print "\t %d" % i, mat_t[i,j]
                if mat_t[i,j]==0:
                    continue
                if mat_t[i,j] == 1:
                    path += [data[j]['path']]
                    break
                if mat_t[i,j] == 2:
                    path += ['identify']
                    break
                if mat_t[i,j] == 3:
                    #path += [data[j]['path']]
                    break
                    pass
                if mat_t[i,j] == 4:
                    path += [u'Requested Treatment']
                    break
                if mat_t[i,j] == 5:
                    path += [u'Requested Callback']
                    break
                if mat_t[i,j] == 6:
                    #print data[j]
                    path += [data[j]['properties']['button_type']]
                    break                  
            if i==0:
                #print path, i, j
                try:
                    ref = re.match(r, data[j]['referrer']).groupdict()['path'] 
                except:
                    ref = data[j]['referrer']
                if data[j]['type']==1:
                    path+=[ref]
                    if j==1:
                        try:
                            path+=[re.match(r, data[0]['referrer']).groupdict()['path'] ]
                        except:
                            path+=[data[0]['referrer']]
                else:
                    path+=[data[j]['path']]
                    path+=[ref]
            j=i
            #print "\t \t" ,i,j, b
            #print (i!=0 or j!=1), '=', (i!=0), (j!=1)
        paths += [path]
    return paths

In [4]:
def filter_paths(paths_t):
    inds=[]
    paths=sorted(paths_t, key=lambda x: len(x),reverse=True)
    for i in range(len(paths)-1,-1,-1):
        for j in range(i-1,-1,-1):
            l = len(paths[j]) - len(paths[i])
            if paths[j][l:]==paths[i]:
                inds+=[i]
                break
    for i in inds:
        tenp=paths.pop(i)
    return paths

In [43]:
def paths_to_str(paths_t):
    str_paths = []
    for i in paths_t:
        str_paths += [' , '.join(list(reversed(i)))]
    return str_paths

In [5]:
files = [os.path.basename(x) for x in glob.glob('./reg_users/*.json')]

In [33]:
f = open('./un_users/%s'%files[31])
context = f.readlines()
data = []
for i in context[:]:
    try:
        data += [json.loads(i.replace(',\n','').replace('\n',''))]
    except:
        pass
data=sorted(data, key=lambda x: x['timestamp'])

In [57]:
all_paths = []
output = open('paths.txt', 'w')
for file_name in files[:1000]:
    f = open('./un_users/%s'%file_name)
    context = f.readlines()
    data = []
    for i in context[:]:
        try:
            data += [json.loads(i.replace(',\n','').replace('\n',''))]
        except:
            pass
    data=sorted(data, key=lambda x: x['timestamp'])
    for path in paths_to_str(filter_paths(get_paths(get_relation_mat(data)))):
        output.write(path.encode('utf-8') + '\n')
        all_paths += [path]
output.close()

In [56]:
path.encode('utf-8')

'http://yandex.ru/clck/ , /clinic/mezhdunarodnuj-medicinskij-centr-\xc2\xabvivantes\xc2\xbb-berlin/'

In [58]:
len(all_paths)

2410

In [59]:
len(set(all_paths))

2278

In [60]:
import re